In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Import TensorFlow
import tensorflow as tf

# 헬퍼 라이브러리들
import numpy as np
import os

print(tf.__version__)

2.2.0


In [2]:
fashion_mnist = tf.keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()


# 하나의 차원을 배열에 추가 -> 새로운 shape == (28, 28, 1)
# 이렇게 하는 이유는 우리의 모델에서 첫 번째 층이 합성곱 층이고
# 합성곱 층은 4D 입력을 요구하기 때문입니다.
# (batch_size, height, width, channels).
# batch_size 차원은 나중에 추가할 것입니다.

train_images = train_images[..., None]
test_images = test_images[..., None]

# 이미지를 [0, 1] 범위로 변경하기.
train_images = train_images / np.float32(255)
test_images = test_images / np.float32(255)

변수와 그래프를 분산하는 전략 만들기
    
- 모든 변수와 모델 그래프는 장치에 복제됩니다.
- 입력은 장치에 고르게 분배되어 들어갑니다.
- 각 장치는 주어지는 입력에 대해서 손실과 그래디언트를 계산합니다.
- 그레디언트들을 전부 더함으로써 모든 장치들 간에 그레디언트들이 동기화됩니다.
- 동기화된 후에, 동일한 업데이트가 각 장치에 있는 변수의 복사본에 동일하게 적용됩니다.

In [3]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


In [4]:
print('device num : {}'.format(strategy.num_replicas_in_sync))

device num : 1


입력 파이프라인 설정

- 그래프와 변수를 플랫폼과 무관한 SavedModel 형식으로 내보냅니다. 모델을 내보냈다면 모델을 불러올 때 범위를 지정해도 되고 하지 않아도 됩니다.

In [5]:
BUFFER_SIZE = len(train_images)

BATCH_SIZE_PER_REPLICA = 64
GLOBAL_BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

EPOCHS = 10

In [6]:
with strategy.scope():
    train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(BUFFER_SIZE).batch(GLOBAL_BATCH_SIZE) 
    train_dist_dataset = strategy.experimental_distribute_dataset(train_dataset)
    
    test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(GLOBAL_BATCH_SIZE) 
    test_dist_dataset = strategy.experimental_distribute_dataset(test_dataset)

In [7]:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Conv2D(64, 3, activation='relu'),
        tf.keras.layers.MaxPooling2D(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    return model

In [8]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')

손실 함수 정의하기

- 일반적으로 gpu/cpu 비율이 1인 단일 장치에서 손실은 입력 배치의 샘플 개수로 나눠집니다.
- 그렇다면 tf.distribute.strategy를 사용할 때 손실은 어떻게 계산되어야 할까요?
  - 예를들면, 4개의 gpu가 있고 입력 배치 크기가 64라고 하죠. 입력 배치 하나가 여러 개의 장치 (4개의 gpu)에 분배됩니다. 각 장치는 크기가 16인 입력을 받습니다.
  - 각 장치에 있는 모델은 해당 입력에 대해 정방향 계산(forward pass)을 수행하고 손실을 계산합니다. 손실을 장치에 할당된 입력 샘플의 수 (batch_size_per_replica=16)로 나누는 것이 아니라 global_batch_size(64)로 나눠야 합니다. 
 - 왜 이렇게 할까요?
   - 위와 같이 계산하는 이유는 그레디언트들이 각 장치에서 계산된 다음, 모든 장치를 동기화하기 위해 이 그레디언트 값들을 전부 더하기 때문입니다.
 - 이것을 텐서플로에서는 어떻게 할까요?
   - 만약 이 예제처럼 사용자 정의 훈련 루프를 작성한다면, 다음과 같이 샘플 당 손실을 더하고 global_batch_size로 나눠야 합니다. scale_loss = tf.reduce_sum(loss) * (1. / global_batch_size) 또는 tf.nn.compute_average_loss 함수를 사용할 수도 있습니다. 이 함수는 샘플 당 손실과 선택적으로 샘플 가중치, global_batch_size를 매개변수 값으로 받고 스케일이 조정된 손실을 반환합니다.
   - 만약 규제 손실을 사용하는 모델이라면, 장치 개수로 손실 값을 소케일 조정해야합니다. 이는 tf.nn_scale_regularization_loss 함수를 사용하여 처리 할 수 있습니다.
   - tf.reduce_mean 을 사용하는 것은 추천하지 않습니다. 이렇게 하는 것은 손실을 실제 장치당 배치 크기로 나눕니다. 이 실제 장치당 배치 크기는 아마 각 단계마다 크기가 다를 수 있습니다.
   - 이런 축소(reduction)와 스케일 조정은 케라스의 model.compile과 model.fit에서 자동적으로 수행됩니다.
   - 만약 tf.keras.losses 클래스를 사용한다면, reduction 매개변수를 명시적으로 NONE 또는 SUM 중 하나로 표시해야 합니다. AUTO가 허용되지 않는 것은 사용자가 분산 모드에서 어떻게 축소할지 명시적으로 지정하는 것이 바람직하기 때문입니다. 현재 SUM_OVER_BATCH_SIZE가 장치당 배치 크기로만 나누고 장치 개수로 나누는 것은 사용자에게 위임하기 때문입니다. 그래서 이렇게 하는 것 대신에 사용자가 명시적으로 축소를 수행하도록 만드는게 좋습니다. 

In [9]:
with strategy.scope():
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(reduction=tf.keras.losses.Reduction.NONE)
    def compute_loss(labels, predictions):
        per_example_loss = loss_object(labels, predictions)
        return tf.nn.compute_average_loss(per_example_loss, global_batch_size=GLOBAL_BATCH_SIZE)

손실과 정확도를 기록하기 위한 지표 정의하기
- 이 지표는 테스트 손실과 훈련 정확도, 테스트 정확도를 기록합니다.
- .result()를 사용해서 누적된 통계값들을 언제나 볼 수 있습니다.

In [10]:
with strategy.scope():
    test_loss = tf.keras.metrics.Mean(name='test_loss')
    train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
    test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [11]:
with strategy.scope():
    model = create_model()
    optimizer = tf.keras.optimizers.Adam()
    checkpoint = tf.train.Checkpoint(optimizer=optimizer, model=model)

In [12]:
with strategy.scope():
    def train_step(inputs):
        images, labels = inputs
        
        with tf.GradientTape() as tape:
            predictions = model(images, training=True)
            loss = compute_loss(labels, predictions)
            
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        train_accuracy.update_state(labels, predictions)
        return loss 

    def test_step(inputs):
        images, labels = inputs
        
        predictions = model(images, training=False)
        t_loss = loss_object(labels, predictions)
        
        test_loss.update_state(t_loss)
        test_accuracy.update_state(labels, predictions)

In [13]:
with strategy.scope():
    
    @tf.function
    def distributed_train_step(dataset_inputs):
        per_replica_losses = strategy.experimental_run_v2(train_step, args=(dataset_inputs,))
        return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses, axis=None)
    
    @tf.function
    def distributed_test_step(dataset_inputs):
        return strategy.experimental_run_v2(test_step, args=(dataset_inputs,))
      
    for epoch in range(EPOCHS):
        
        total_loss = 0.0
        num_batches = 0
        for x in train_dist_dataset:
            total_loss += distributed_train_step(x)
            num_batches += 1
        train_loss = total_loss / num_batches

        for x in test_dist_dataset:
            distributed_test_step(x)

        if epoch % 2 == 0:
            checkpoint.save(checkpoint_prefix)

        template = ("에포크 {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}")
        print (template.format(epoch+1, train_loss, train_accuracy.result()*100, test_loss.result(), test_accuracy.result()*100))

        test_loss.reset_states()
        train_accuracy.reset_states()
        test_accuracy.reset_states()

Instructions for updating:
renamed to `run`
에포크 1, 손실: 0.5101363658905029, 정확도: 81.59666442871094, 테스트 손실: 0.4765279293060303, 테스트 정확도: 82.83000183105469
에포크 2, 손실: 0.3379530906677246, 정확도: 87.75167083740234, 테스트 손실: 0.32694604992866516, 테스트 정확도: 87.86000061035156
에포크 3, 손실: 0.29186832904815674, 정확도: 89.29332733154297, 테스트 손실: 0.3060758709907532, 테스트 정확도: 88.8499984741211
에포크 4, 손실: 0.2608529031276703, 정확도: 90.48332977294922, 테스트 손실: 0.3086162507534027, 테스트 정확도: 88.81999969482422
에포크 5, 손실: 0.23396387696266174, 정확도: 91.38166809082031, 테스트 손실: 0.2818750739097595, 테스트 정확도: 89.92000579833984
에포크 6, 손실: 0.21336181461811066, 정확도: 92.20832824707031, 테스트 손실: 0.2568416893482208, 테스트 정확도: 90.51000213623047
에포크 7, 손실: 0.19619984924793243, 정확도: 92.73832702636719, 테스트 손실: 0.2622269093990326, 테스트 정확도: 90.62000274658203
에포크 8, 손실: 0.17784860730171204, 정확도: 93.4800033569336, 테스트 손실: 0.26398640871047974, 테스트 정확도: 90.6500015258789
에포크 9, 손실: 0.1636963188648224, 정확도: 93.97333526611328, 테스트 손실: 0.2520776

- 이 예제는 train_dist_dataset과 test_dist_dataset을 for x in ... 구조를 통해서 반복합니다.
- 스케일이 조정된 손실은 distributed_train_step의 반환값입니다. tf.distribute.Strategy.reduce 호출을 사용해서 장치들 간의 스케일이 조정된 손실 값을 전부 합칩니다. 그리고 나서 tf.distribute.Strategy.reduce 반환 값을 더하는 식으로 배치 간의 손실을 모읍니다.
- tf.keras.Metrics는 tf.distribute.Strategy.experimental_run_v2 에 의해서 실행되는 train_step, test_step 함수 안에서 업데이트 되어야 합니다.
- tf.distribute.Strategy.experimental_run_v2는 그 전략 안에 포함된 각 지역 복제 모델로부터 결과값을 반환해줍니다. 그리고 이 결과를 사용하는 몇 가지 방법들이 있습니다. tf.distribute.Strategy.reduce를 이용하여 값들을 합칠 수 있습니다. tf.distribute.Strategy.experimental_local_results를 사용해서 결과값(지역 복제 모델 당 하나의 결과값)에 들어있는 값들의 리스트를 얻을 수도 있습니다.

최신 체크포인트를 불러와서 테스트하기
- tf.distribute.Strategy를 사용해서 체크포인트가 만들어진 모델은 Strategy 사용 여부에 상관없이 불러올 수 있습니다.

In [14]:
eval_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='eval_accuracy')
new_model = create_model()
new_optimizer = tf.keras.optimizers.Adam()

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(GLOBAL_BATCH_SIZE)

In [15]:
@tf.function
def eval_step(images, labels):
    predictions = new_model(images, training=False)
    eval_accuracy(labels, predictions)

In [16]:
checkpoint = tf.train.Checkpoint(optimizer=new_optimizer, model=new_model)
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

for images, labels in test_dataset:
    eval_step(images, labels)

print('전략을 사용하지 않고, 저장된 모델을 복원한 후의 정확도:{}'.format(eval_accuracy.result()*100))

전략을 사용하지 않고, 저장된 모델을 복원한 후의 정확도:91.29999542236328


데이터셋에 대해 반복 작업을 하는 다른 방법들
반복자(iterator)를 사용하기
- 만약 주어진 스텝의 수에 따라서 반복하기 원하면서 전체 데이터셋을 보는 것을 원치 않는다면, iter를 호출하여 반복자를 만들 수 있습니다. 그 다음 명시적으로 next를 호출합니다. 또한, tf.function 내부 또는 외부에서 데이터셋을 반복하도록 설정 할 수 있습니다. 다음은 반복자를 사용하여 tf.function 외부에서 데이터셋을 반복하는 코드 예제입니다.

In [17]:
with strategy.scope():
    for _ in range(EPOCHS):
        total_loss = 0.0
        num_batches = 0
        train_iter = iter(train_dist_dataset)
        
        for _ in range(10):
            total_loss += distributed_train_step(next(train_iter))
            num_batches += 1
        average_train_loss = total_loss / num_batches
        
        template = ('epoch {}, loss: {}, accuracy: {}')
        print(template.format(epoch+1, average_train_loss, train_accuracy.result()*100))
        train_accuracy.reset_states()

epoch 10, loss: 0.13899457454681396, accuracy: 95.3125
epoch 10, loss: 0.15233740210533142, accuracy: 94.375
epoch 10, loss: 0.15641267597675323, accuracy: 94.375
epoch 10, loss: 0.13490642607212067, accuracy: 95.15625
epoch 10, loss: 0.16956572234630585, accuracy: 94.53125
epoch 10, loss: 0.09844110906124115, accuracy: 96.40625
epoch 10, loss: 0.10222840309143066, accuracy: 96.5625
epoch 10, loss: 0.1697273701429367, accuracy: 92.96875
epoch 10, loss: 0.11285873502492905, accuracy: 96.25
epoch 10, loss: 0.11263132095336914, accuracy: 95.9375


tf.function 내부에서 반복하기
- 전체 입력 train_dist_dataset 에 대해서 tf.function 내부에서 for x in ... 생성자를 사용함으로써 반복을 하거나, 위에서 사용했던 것처럼 반복자를 사용함으로써 반복을 할 수 있습니다. 아래의 예제에서는 tf.function 로 한 훈련의 에포크를 감싸고 그 함수에서 train_dist_dataset 를 반복하는 것을 보여줍니다. 

In [18]:
with strategy.scope():
    @tf.function
    def distributed_train_epoch(dataset):
        total_loss = 0.0
        num_batches = 0
        for x in dataset:
            per_replica_losses = strategy.experimental_run_v2(train_step, args=(x,))
            total_loss += strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses, axis=None)
            num_batches += 1
        return total_loss / tf.cast(num_batches, dtype=tf.float32)
    
    for epoch in range(EPOCHS):
        train_loss = distributed_train_epoch(train_dist_dataset)
        template = ('Epoch: {}, Loss: {}, Accuracy: {}')
        print(template.format(epoch+1, train_loss, train_accuracy.result()*100))
        
        train_accuracy.reset_states()

Epoch: 1, Loss: 0.13619457185268402, Accuracy: 94.95333099365234
Epoch: 2, Loss: 0.12229454517364502, Accuracy: 95.46833038330078
Epoch: 3, Loss: 0.11396694928407669, Accuracy: 95.79666900634766
Epoch: 4, Loss: 0.10390675812959671, Accuracy: 96.08833312988281
Epoch: 5, Loss: 0.09489856660366058, Accuracy: 96.45000457763672
Epoch: 6, Loss: 0.08482521027326584, Accuracy: 96.75166320800781
Epoch: 7, Loss: 0.07919544726610184, Accuracy: 97.08333587646484
Epoch: 8, Loss: 0.07550602406263351, Accuracy: 97.23332977294922
Epoch: 9, Loss: 0.06530527025461197, Accuracy: 97.56666564941406
Epoch: 10, Loss: 0.061437640339136124, Accuracy: 97.70667266845703
